In [7]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [3]:
llm=ChatGroq(model="openai/gpt-oss-120b")

In [4]:
llm.invoke("hi, how are you?")

AIMessage(content="Hey there! I'm doing great, thanks for asking. How can I help you today?", additional_kwargs={'reasoning_content': 'We need to respond. The system messages say we are ChatGPT, with normal behavior. The user asks "hi, how are you?" We can respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 77, 'total_tokens': 138, 'completion_time': 0.129125104, 'prompt_time': 0.002871345, 'queue_time': 0.003780085, 'total_time': 0.131996449}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_213abb2467', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fd14783a-4d21-411e-a61e-769bb780f73b-0', usage_metadata={'input_tokens': 77, 'output_tokens': 61, 'total_tokens': 138})

In [8]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [9]:
print(multiply)

name='multiply' description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x0000021FA9311E40>


In [10]:
class WeatherInput(BaseModel):
    city: str

In [ ]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [ ]:
weather_tool = StructuredTool.from_function(
    func = get_weather,
    name = "get_weather",
    description = "Fetches real-time weather data for a city",
    args_schema = WeatherInput,  
)

In [ ]:
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

In [ ]:
class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = (
        "Fetches weather data for a city"
    )
    
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        return get_weather(city, units)